In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.2 MB 28.0 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 


In [2]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [3]:
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

수요-공급 최적화문제:
1. 모델 선언 ( model = ConcreteModel )
2. Inventory 데이터 정의
3. 변수 정의 (binary, integer 등등)
4. 목적 함수 정의 (최소화 or 최대화)
5. 제약조건 정의
6. Solver 정의 및 결과 해석

# Knapsack Problem

In [7]:
from pyomo.environ import *

# Inventory Data  정의
A = ['hammer', 'wrench', 'screwdriver','towel']    # item 목록
b = {'hammer':8, 'wrench':3, 'screwdriver':6,'towel':11}  # benefit
w = {'hammer':5, 'wrench':7, 'screwdriver':4,'towel':3}   # 각 item들의 무게
W_max = 14 # 무게 제한

model = ConcreteModel()         # 모델 생성
model.x = Var( A, within= Binary) # 각 아이템들 binary 로 정의 (사용 할지 안할지)

#목적함수 정의 benefit을 최대화
#각 아이템들 benefit 합산
model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A),
    sense = maximize  
)

#제약조건 아이템들의 무게 합계가 무게 제한  W_max=14 보다 적어야함.
model.weight_con = Constraint(
    expr = sum( w[i] * model.x[i] for i in A) <= W_max
)

# solver 정의 및 최적화 문제 풀기
opt = SolverFactory('glpk')
opt_success = opt.solve(model)

# model.display
model.x.pprint()

x : Size=4, Index=x_index
    Key         : Lower : Value : Upper : Fixed : Stale : Domain
         hammer :     0 :   1.0 :     1 : False : False : Binary
    screwdriver :     0 :   1.0 :     1 : False : False : Binary
          towel :     0 :   1.0 :     1 : False : False : Binary
         wrench :     0 :   0.0 :     1 : False : False : Binary


# Warehouse Problem

In [10]:
from typing import Container
from pyomo.environ import *

# 모델생성
model = ConcreteModel(name="(WL")

#Inventory 정의
# 창고 (warehouse) 이름 정의
W = ['Harlingen', 'Memphis', 'Ashland']
# 도시 이름 정의   
C = ['NYC', 'LA', 'Chicago', 'Houston']    
# 도시간 제공 비용 w - > c
d = {('Harlingen', 'NYC'): 1956, \
     ('Harlingen', 'LA'): 1606, \
     ('Harlingen', 'Chicago'): 1410, \
     ('Harlingen', 'Houston'): 330, \
     ('Memphis', 'NYC'): 1096, \
     ('Memphis', 'LA'): 1792, \
     ('Memphis', 'Chicago'): 531, \
     ('Memphis', 'Houston'): 567, \
     ('Ashland', 'NYC'): 485, \
     ('Ashland', 'LA'): 2322, \
     ('Ashland', 'Chicago'): 324, \
     ('Ashland', 'Houston'): 1236 }
# 창고 W개 중 최대 창고 개수 제한 (2개) 
P = 2

# 변수 정의 
model.x = Var(W, C, bounds=(0,1))
model.y = Var(W, within=Binary)

#목적 함수 정의
def obj_rule(m):
    return sum(d[w,c]*m.x[w,c] for w in W for c in C)
model.obj = Objective(rule=obj_rule)

#제약조건 정의
@model.Constraint (C)
def one_per_cust_rule(m, c):
    return sum(m.x[w,c] for w in W) == 1

@model.Constraint (W,C)
def warehouse_active_rule(m, w, c):
    return m.x[w,c] <= m.y[w]

@model.Constraint
def num_warehouses_rule(m):
    return sum(m.y[w] for w in W) <= P

#문제 풀이
SolverFactory('glpk').solve(model)

model.y.pprint()
model.x.pprint()

y : Size=3, Index=y_index
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
      Ashland :     0 :   1.0 :     1 : False : False : Binary
    Harlingen :     0 :   1.0 :     1 : False : False : Binary
      Memphis :     0 :   0.0 :     1 : False : False : Binary
x : Size=12, Index=x_index
    Key                      : Lower : Value : Upper : Fixed : Stale : Domain
      ('Ashland', 'Chicago') :     0 :   1.0 :     1 : False : False :  Reals
      ('Ashland', 'Houston') :     0 :   0.0 :     1 : False : False :  Reals
           ('Ashland', 'LA') :     0 :   0.0 :     1 : False : False :  Reals
          ('Ashland', 'NYC') :     0 :   1.0 :     1 : False : False :  Reals
    ('Harlingen', 'Chicago') :     0 :   0.0 :     1 : False : False :  Reals
    ('Harlingen', 'Houston') :     0 :   1.0 :     1 : False : False :  Reals
         ('Harlingen', 'LA') :     0 :   1.0 :     1 : False : False :  Reals
        ('Harlingen', 'NYC') :     0 :   0.0 :     1 : False : False : 

# Nonlinear Rosenberg Problem

In [11]:
from pyomo.core.base import initializer
#비선형 최적화

from pyomo.environ import *

# 모델정의
model = ConcreteModel()

# 변수 및 초기 조건 정의
model.x = Var(initialize=1.5)
model.y = Var(initialize=1.5)

# 함수 정의 (최소화)
def rosenbrock(model):
  return (1.0-model.x)**2 + 100*(model.y - model.x**2)**2
model.obj = Objective(rule = rosenbrock, sense = minimize)

SolverFactory('ipopt').solve(model)
#모델결과 불러오기
model.x.pprint()
model.y.pprint()

x : Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :  None : 1.0000000000008233 :  None : False : False :  Reals
y : Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :  None : 1.0000000000016314 :  None : False : False :  Reals
